In [68]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
import nbformat
import logging

ROOT = Path("..").resolve()
sys.path.append(str(ROOT / "src"))

from bootcamp_data.config import make_paths

paths = make_paths(ROOT)
FIGS = ROOT / "reports" / "figures"
FIGS.mkdir(parents=True, exist_ok=True)

def save_fig(fig, filename):
    path = FIGS / filename
    try:
        fig.write_image(str(path), scale=2)
        print(f"Saved: {path}")
    except:
        print("Kaleido not found, skipping export.")

In [69]:
df = pd.read_parquet(paths.processed / "analytics_table.parquet")

print(f"Rows: {len(df)}")
print(df.dtypes.head(15))
print("\nTop missing columns:")
print(df.isna().sum().sort_values(ascending=False).head())

Rows: 100
order_id               string[python]
user_id                string[python]
amount                        Float64
quantity                        Int64
created_at        datetime64[ns, UTC]
status                         object
status_clean                   object
amount _isna                     bool
quantity _isna                   bool
date                           object
year                          float64
month                  string[python]
dow                            object
hour                          float64
country                        object
dtype: object

Top missing columns:
amount_winsor        12
amount               12
amount_is_outlier    12
quantity              8
hour                  7
dtype: int64


# EDA Plan & Questions

We will answer the following questions to understand sales performance and data quality:

1. **Trend:** How does total revenue change over time (by month)?
2. **Geography:** Which country generates the most revenue?
3. **Distribution:** What is the distribution of order amounts (winsorized)?
4. **Risk Analysis:** Which country has the highest refund rate?
5. **Customer Behavior:** What is the busiest day of the week for orders?


In [76]:
df['week'] = df['created_at'].dt.tz_localize(None).dt.to_period('W').astype(str)

q1_df = (
    df.groupby("week", dropna=False)
    .agg(
        total_revenue=("amount", "sum"), 
        orders_count=("order_id", "count")
    )
    .reset_index()
    .sort_values("week")
)

print("--- Weekly Revenue Table ---")
print(q1_df.sort_values("total_revenue", ascending=False).head())

fig1 = px.line(
    q1_df, 
    x="week", 
    y="total_revenue", 
    markers=True,
    title="Q1: Weekly Revenue Trend"
)
fig1.update_layout(title_x=0.5)
fig1.show()
save_fig(fig1, "q1_weekly_trend.png")

--- Weekly Revenue Table ---
                    week  total_revenue  orders_count
4  2025-12-29/2026-01-04          487.0            14
6  2026-01-12/2026-01-18          487.0            13
2  2025-12-15/2025-12-21         480.99            12
5  2026-01-05/2026-01-11         478.89            14
3  2025-12-22/2025-12-28         431.49            14


Saved: C:\Users\faisal_333\bootcamp\week2-data-work\reports\figures\q1_weekly_trend.png


**Interpretation:**
- The highest revenue was recorded in the weeks of **2025-12-29/2026-01-04 and 2026-01-12/2026-01-18**.
- There is significant volatility between weeks; sales are not stable.
- **Caveat:** Some weeks might look low simply because we have incomplete data for that specific week (data collection issues).

**Key Findings:**
*   **Stability:** Revenue is generally stable between 300 and 500 per week.
*   **Peak:** The highest revenue occurred in mid-January (~490).
*   **Caveat (Dip):** The sharp drop in the last week (to ~100) likely indicates incomplete data for that week (partial week), not a sales crash.

In [ ]:
q2_df = (
    df.groupby("country", dropna=False)
    .agg(
        total_revenue=("amount", "sum"),
        orders_count=("amount", "count")  
    )
    .reset_index()
    .sort_values("total_revenue", ascending=False)
)

q2_df["AOV"] = q2_df["total_revenue"] / q2_df["orders_count"]

print("--- Country Performance ---")
display(q2_df)

fig2 = px.bar(
    q2_df,
    x="country",
    y="total_revenue",
    text_auto=True,
    title="Q2: Total Revenue by Country"
)
fig2.update_layout(title_x=0.5)
fig2.show()
save_fig(fig2, "q2_revenue_country.png")

--- Country Performance ---


,country,total_revenue,orders_count,AOV
1,SA,2753.23,68,40.488676
0,AE,649.48,20,32.474


Saved: C:\Users\faisal_333\bootcamp\week2-data-work\reports\figures\q2_revenue_country.png


**Key Findings:**
*   **Dominance:** Saudi Arabia (SA) is the primary market, generating ~2,750 in revenue compared to ~650 for UAE (AE).
*   **Volume:** SA orders volume is roughly 4x larger than AE.
*   **Action:** Marketing efforts should prioritize maintaining the SA market share while investigating growth opportunities in AE.

In [ ]:
fig3 = px.histogram(
    df, 
    x="amount_winsor", 
    nbins=20,
    title="Q3: Distribution of Order Amounts (Winsorized)"
)
print("--- Amount Statistics (Winsorized) ---")
display(df["amount_winsor"].describe().to_frame())

fig3.update_layout(title_x=0.5, xaxis_title="Amount (Capped)")
fig3.show()
save_fig(fig3, "q3_amount_dist.png")


--- Amount Statistics (Winsorized) ---


,amount_winsor
count,88.0
mean,38.716591
std,40.428404
min,4.35
25%,12.875
50%,23.05
75%,46.25
max,200.0


Saved: C:\Users\faisal_333\bootcamp\week2-data-work\reports\figures\q3_amount_dist.png


**Key Findings:**
*   **Skewness:** The distribution is heavily right-skewed; most orders are small values (under 50).
*   **Typical Order:** The median order value is much lower than the mean, indicating that a few large orders pull the average up.
*   **Range:** The majority of orders fall between 10 and 50 (based on the histogram bins).

In [ ]:
df["is_refund"] = (df["status_clean"] == "refund").astype(int)

q4_df = (
    df.groupby("country", dropna=False)
    .agg(
        refund_count=("is_refund", "sum"),
        total_orders=("order_id", "count")
    )
    .reset_index()
)
q4_df["refund_rate"] = q4_df["refund_count"] / q4_df["total_orders"]

print("--- Refund Risks ---")
display(q4_df.sort_values("refund_rate", ascending=False))

fig4 = px.bar(
    q4_df,
    x="country",
    y="refund_rate",
    text_auto=".1%", 
    title="Q4: Refund Rate by Country (Risk Analysis)"
)
fig4.update_yaxes(tickformat=".0%")
fig4.update_layout(title_x=0.5)
fig4.show()
save_fig(fig4, "q4_refund_rate.png")

--- Refund Risks ---


,country,refund_count,total_orders,refund_rate
0,AE,5,24,0.208333
1,SA,9,76,0.118421


Saved: C:\Users\faisal_333\bootcamp\week2-data-work\reports\figures\q4_refund_rate.png


**Key Findings:**
*   **High Risk in AE:** The UAE (AE) has a significantly higher refund rate (~20.8%) compared to SA (~11.8%).
*   **Investigation Needed:** Although SA has more volume, almost 1 in 5 orders in AE ends up as a refund. We need to investigate logistics or product quality specific to the UAE region.

In [ ]:
days_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

q5_df = (
    df.groupby("dow", dropna=False)
    .agg(orders_count=("order_id", "count"))
    .reset_index()
)
print("--- Daily Volume ---")
display(q5_df.sort_values("orders_count", ascending=False))

fig5 = px.bar(
    q5_df, 
    x="dow", 
    y="orders_count",
    category_orders={"dow": days_order}, 
    title="Q5: Orders Volume by Day of Week"
)

fig5.update_layout(title_x=0.5)
fig5.show()
save_fig(fig5, "q5_daily_volume.png")

--- Daily Volume ---


,dow,orders_count
1,Monday,15
3,Sunday,14
6,Wednesday,14
0,Friday,13
5,Tuesday,13
2,Saturday,12
4,Thursday,12
7,NaN,7


Saved: C:\Users\faisal_333\bootcamp\week2-data-work\reports\figures\q5_daily_volume.png


**Key Findings:**
*   **Peak Day:** Monday is the busiest day with 15 orders.
*   **Consistency:** The volume is relatively consistent across the week (ranging from 12 to 15 orders), with no massive drops on weekends.
*   **Staffing:** Support or operations teams should ensure full coverage on Mondays.
